In [0]:
import os
import sys
import tensorflow as tf
import numpy as np
import math
import timeit
import scipy

#import matplotlib.pyplot as plt

import provider
import TR
#%matplotlib inline

In [0]:
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

In [0]:
from mobilenet_v3_block import BottleNeck, h_swish

class CentralNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2,channel_3,channel_4, NUM_CLASSES,c1,c2,c3,c4):
        super().__init__()
        initializer = tf.variance_scaling_initializer(scale=2.0)

        # mobilenet
        self.conv1 = tf.keras.layers.Conv2D(filters=16,kernel_size=(3, 3),strides=2,padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bneck1 = BottleNeck(in_size=16, exp_size=16, out_size=16, s=2, is_se_existing=True, NL="RE", k=3)
        self.bneck2 = BottleNeck(in_size=16, exp_size=72, out_size=24, s=2, is_se_existing=False, NL="RE", k=3)
        self.bneck3 = BottleNeck(in_size=24, exp_size=88, out_size=24, s=1, is_se_existing=False, NL="RE", k=3)
        self.bneck4 = BottleNeck(in_size=24, exp_size=96, out_size=40, s=2, is_se_existing=True, NL="HS", k=5)
        self.bneck5 = BottleNeck(in_size=40, exp_size=240, out_size=40, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck6 = BottleNeck(in_size=40, exp_size=240, out_size=40, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck7 = BottleNeck(in_size=40, exp_size=120, out_size=48, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck8 = BottleNeck(in_size=48, exp_size=144, out_size=48, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck9 = BottleNeck(in_size=48, exp_size=288, out_size=96, s=2, is_se_existing=True, NL="HS", k=5)
        self.bneck10 = BottleNeck(in_size=96, exp_size=576, out_size=96, s=1, is_se_existing=True, NL="HS", k=5)
        self.bneck11 = BottleNeck(in_size=96, exp_size=576, out_size=96, s=1, is_se_existing=True, NL="HS", k=5)

        self.conv2 = tf.keras.layers.Conv2D(filters=576,kernel_size=(1, 1),strides=1,padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.avgpool = tf.keras.layers.AveragePooling2D(pool_size=(1, 1),strides=1)
        self.conv3 = tf.keras.layers.Conv2D(filters=1280,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.conv4 = tf.keras.layers.Conv2D(filters=NUM_CLASSES,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same",
                                            activation=tf.keras.activations.softmax)
        # fusion
                                     
        self.fc4c1 = tf.layers.Dense(c1, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c14c2 = tf.layers.Dense(c2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c2 = tf.layers.Dense(c2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c24c3 = tf.layers.Dense(c3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c3 = tf.layers.Dense(c3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c34c4 = tf.layers.Dense(c4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c4 = tf.layers.Dense(c4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c44c5 = tf.layers.Dense(1280, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.flatten = tf.keras.layers.Flatten()
        
        self.fc1=tf.layers.Dense(1280, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc2=tf.layers.Dense(1280, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc3=tf.layers.Dense(576, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc4=tf.layers.Dense(num_classes, activation = None, 
                        kernel_initializer=initializer)
     
    def call(self, x1,x2, training=None):
        scores = None
        ###x1

        x1 = self.conv1(x1)
        x1 = self.bn1(x1, training=training)
        x1 = h_swish(x1)

        x1 = self.bneck1(x1, training=training)
        x1 = self.bneck2(x1, training=training)
        x1 = self.bneck3(x1, training=training)
        h1_x1=self.flatten(x1)
        x1 = self.bneck4(x1, training=training)
        x1 = self.bneck5(x1, training=training)
        x1 = self.bneck6(x1, training=training)
        h2_x1=self.flatten(x1)
        x1 = self.bneck7(x1, training=training)
        x1 = self.bneck8(x1, training=training)
        h3_x1=self.flatten(x1)
        x1 = self.bneck9(x1, training=training)
        x1 = self.bneck10(x1, training=training)
        x1 = self.bneck11(x1, training=training)
        h4_x1=self.flatten(x1)
        x1 = self.conv2(x1)
        x1 = self.bn2(x1, training=training)
        x1 = h_swish(x1)
        x1 = self.avgpool(x1)
        x1 = self.conv3(x1)
        x1 = h_swish(x1)
        x1 = self.conv4(x1)
        h5_x1=self.flatten(x1)
        ###x2

        x2 = self.conv1(x2)
        x2 = self.bn1(x2, training=training)
        x2 = h_swish(x2)

        x2 = self.bneck1(x2, training=training)
        x2 = self.bneck2(x2, training=training)
        x2 = self.bneck3(x2, training=training)
        h1_x2=self.flatten(x2)
        x2 = self.bneck4(x2, training=training)
        x2 = self.bneck5(x2, training=training)
        x2 = self.bneck6(x2, training=training)
        h2_x2=self.flatten(x2)
        x2 = self.bneck7(x2, training=training)
        x2 = self.bneck8(x2, training=training)
        h3_x2=self.flatten(x2)
        x2 = self.bneck9(x2, training=training)
        x2 = self.bneck10(x2, training=training)
        x2 = self.bneck11(x2, training=training)
        h4_x2=self.flatten(x2)
        x2 = self.conv2(x2)
        x2 = self.bn2(x2, training=training)
        x2 = h_swish(x2)
        x2 = self.avgpool(x2)
        x2 = self.conv3(x2)
        x2 = h_swish(x2)
        x2 = self.conv4(x2)
        h5_x2=self.flatten(x2)

        hc1=self.fc4c1(h1_x1)+self.fc4c1(h1_x2)
        hc2=self.fc4c2(h2_x1)+self.fc4c2(h2_x2)+self.c14c2(hc1)
        hc3=self.fc4c3(h3_x1)+self.fc4c3(h3_x2)+self.c24c3(hc2)
        hc4=self.fc4c4(h4_x1)+self.fc4c4(h4_x2)+self.c34c4(hc3)
        hc5=self.fc1(h5_x1)+self.fc1(h5_x2)+self.c44c5(hc4)
        dense2=self.fc2(hc5) #1280
        dense3=self.fc3(dense2)#1000
        scores= self.fc4(dense3)#num_classes
        return scores


def placeholder_inputs(batch_size, img_rows=66, img_cols=200, points=16384, separately=False):
    imgs_pl = tf.placeholder(tf.float32, shape=(batch_size, img_rows, img_cols, 3))
    pts_pl = tf.placeholder(tf.float32, shape=(batch_size, points, 3))
    if separately:
        speeds_pl = tf.placeholder(tf.float32, shape=(batch_size))
        angles_pl = tf.placeholder(tf.float32, shape=(batch_size))
        labels_pl = [speeds_pl, angles_pl]
    labels_pl = tf.placeholder(tf.float32, shape=(batch_size, 2))
    return imgs_pl, pts_pl, labels_pl


def get_loss(pred, label, l2_weight=0.0001):
    diff = tf.square(tf.subtract(pred, label))
    train_vars = tf.trainable_variables()
    l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in train_vars[1:]]) * l2_weight
    loss = tf.reduce_mean(diff + l2_loss)

    return loss



In [0]:
'''testing the net
tf.reset_default_graph()

channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4=64,128,256,512,10,64,128,256,512
model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4)
with tf.device(device):
    x1 = tf.zeros((64, 32, 32,3))
    x2 = tf.zeros((64, 32, 32,3))
    scores = model(x1,x2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    scores_np = sess.run(scores)
    print(scores_np.shape)
'''

In [0]:
BATCH_SIZE=8
MAX_EPOCH=1
learning_rate = 3e-5
channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4=64,128,256,512,2,64,128,256,512

def model_init_fn(inputs1,inputs2, is_training):
    model = None

    model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4)

    return model(inputs1,inputs2)

def optimizer_init_fn():
    optimizer = None

    optimizer = tf.train.AdamOptimizer()

    return optimizer

TR.train_part34(model_init_fn, optimizer_init_fn,10,device,BATCH_SIZE)